## Big Mart Sales Prediction

### Research Question

Through the analysis, I am planning to predict the impact of other factors on sales of a particular product in a particular store. The train file will be used to explore the data and train the model. The model will be applied on test data in the test file and model performance will be evaluated.

### Data Source

The data has been collected from [here](https://code.datasciencedojo.com/tshrivas/dojoHub/tree/a152a17dee24dcfcc10bb75c77c2e88cdcf90212/Big%20Mart%20Sales%20DataSet)

In [6]:
# import libraries

import pandas as pd
import numpy as np
import pandas_profiling
import warnings

# suppress unnecessary warnings
def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [21]:
# data location

dirData = 'Data/'
fileTrain = 'train.csv'
fileTest = 'test.csv'

# read files

train = pd.read_csv(dirData+fileTrain)
test = pd.read_csv(dirData+fileTest)

Since the source files are already divided into train and test, following machine learning best practices, I decided to merge them, yet keeping an indicator. This way, feature engineering could be applied on the entire dataset prior to training and testing.

In [26]:
# add identifier columns for futureuse
train['source']='train'
test['source']='test'

# merge data
sales_df = pd.concat([train, test],ignore_index=True, sort=False)

# check row count
print('Train: ',train.shape)
print('Test: ',test.shape)
print('Sales: ',sales_df.shape)

# check merged data
sales_df.head()

Train:  (8523, 13)
Test:  (5681, 12)
Sales:  (14204, 13)


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,train
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,train
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,train
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800,train
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,train


### Exploratory Analysis

In [27]:
# create data profile
pandas_profiling.ProfileReport(sales_df)

Number of variables,13
Number of observations,14204
Total Missing (%),6.6%
Total size in memory,1.4 MiB
Average record size in memory,104.0 B
Numeric,5
Categorical,8
Boolean,0
Date,0
Text (Unique),0
Rejected,0


Analysis of each variable in the dataset might be redundant. Therefore, documenting the interesting observations as below:
    
* `Item_Fat_Content`: The variable has _6.6%_ missing values. But what is more important is it needs some data cleansing. The unique values contain 'Low Fat', 'LF', 'low fat' and 'Regular', 'reg'. They need to be encoded correctly in the data cleansing process.

* `Item_Identifier`: There are 1559 distinct values, meaning we have data for 10 different items.

* `Outlet_Identifier`: There are 10 distinct values, meaning we have data for 10 different stores.

* `Item_MRP`: The extreme values range from 31.29 to 266.89. This is a critical component in sales prediction. It is good that there is no missing value.

* `Item_Type`: Item type has 16 unique values, i.e. there are 16 different item types in the data.

* `Item_Visibility`: I was expecting this to be categorical, but visibility is denoted by numeric values. I may or may not change it in future based on subsequent findings.

* `Item_Outlet_Sales`: It is interesting that this variable is missing _40%_ of the values. But a closer look will tell that this is the target variable that we are trying to predict in the test dataset. So the missing values are perfectly ok.

* `Outlet_Size`: Interestingly, this categorical variable is missing _28.3%_ values.

### Data Cleansing